In [20]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import itertools
import json
import pickle
import gensim

In [16]:
# Open everything that's been saved in /data/conditioned

df=pd.read_csv("../../data/conditioned/master-lyricsdf.csv")


with open("../../data/conditioned/noun-n-gram.json") as json_file:
    noungram = json.load(json_file)
with open("../../data/conditioned/adj-n-gram.json") as json_file:
    adjgram = json.load(json_file)
with open("../../data/conditioned/nounvocab.json") as json_file:
    nounvocab = json.load(json_file)
with open("../../data/conditioned/nounid2word.json") as json_file:
    nounid2word = json.load(json_file)
with open("../../data/conditioned/adjvocab.json") as json_file:
    adjvocab = json.load(json_file)
with open("../../data/conditioned/adjid2word.json") as json_file:
    adjid2word = json.load(json_file)


f = open("../../data/conditioned/ahypes.p",'r')  
ahypes = pickle.load(f)  
f = open("../../data/conditioned/nhypes.p",'r')  
nhypes = pickle.load(f)
f = open("../../data/conditioned/corpus.p",'r')  
corpus = pickle.load(f)


df.head()

,position,year,title.href,title,artist,lyrics,decade,song_key,lyrics_url,lyrics_abstract
0,1,1970,https://en.wikipedia.org/wiki/Bridge_over_Trou...,Bridge over Troubled Water,Simon and Garfunkel,When you're weary feeling small When tears are...,1970,1970-1,http://lyrics.wikia.com/Simon_And_Garfunkel:Br...,When you're weary\nFeeling small\nWhen tears a...
1,2,1970,https://en.wikipedia.org/wiki/(They_Long_to_Be...,(They Long to Be) Close to You,The Carpenters,x,1970,1970-2,http://lyrics.wikia.com/Carpenters:%28They_Lon...,Why do birds suddenly appear\nEverytime you ar...
2,3,1970,https://en.wikipedia.org/wiki/American_Woman_(...,American Woman,The Guess Who,"American woman, stay away from me American wom...",1970,1970-3,http://lyrics.wikia.com/The_Guess_Who:American...,"Mmm, da da da\nMmm, mmm, da da da\nMmm, mmm, d..."
3,4,1970,https://en.wikipedia.org/wiki/Raindrops_Keep_F...,Raindrops Keep Fallin' on My Head,B.J. Thomas,Raindrops keep falling on my head Just like th...,1970,1970-4,http://lyrics.wikia.com/B.J._Thomas:Raindrops_...,Raindrops are falling on my head\nAnd just lik...
4,5,1970,https://en.wikipedia.org/wiki/War_(Edwin_Starr...,War,Edwin Starr,"War huh Yeah! Absolutely uh-huh, uh-huh huh Ye...",1970,1970-5,http://lyrics.wikia.com/Edwin_Starr:War,"War, huh, yeah\nWhat is it good for?\nAbsolute..."


In [105]:
id2w=dict()
run = dict()
ntopics=4
for k in nounid2word:
    id2w[int(k)]=nounid2word[k]
# reduce the chunksize and make passes=2 since our data is not so big.
for i in range(1,ntopics):
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2w, num_topics=i+1, update_every=1, chunksize=100, passes=1)
    run[i+1]= lda.print_topics(num_topics=i+1,num_words=10)

In [106]:
for i in range(1,ntopics):
    print "\n Topics: ",i+1
    line=run[i+1]
    for j in range(0,i+1):
        print line[j]
        print "\n"


 Topics:  2
0.014*money + 0.013*nigga + 0.013*bitch + 0.010*man + 0.010*shit + 0.010*body + 0.009*everybody + 0.009*club + 0.009*hitta + 0.009*ya


0.039*baby + 0.034*girl + 0.032*time + 0.030*love + 0.028*night + 0.028*way + 0.020*heart + 0.019*thing + 0.018*life + 0.014*tonight



 Topics:  3
0.042*time + 0.040*love + 0.037*night + 0.029*way + 0.027*heart + 0.024*life + 0.022*thing + 0.018*day + 0.017*tonight + 0.014*hand


0.020*nigga + 0.019*bitch + 0.015*shit + 0.013*party + 0.013*club + 0.013*dance + 0.010*round + 0.009*ass + 0.009*hitta + 0.008*cause


0.065*girl + 0.060*baby + 0.030*ya + 0.025*boy + 0.023*money + 0.021*man + 0.014*home + 0.014*rock + 0.011*way + 0.011*somebody



 Topics:  4
0.030*money + 0.026*nigga + 0.025*bitch + 0.021*man + 0.021*shit + 0.018*club + 0.014*ass + 0.010*fall + 0.010*hitta + 0.009*wit


0.053*girl + 0.049*time + 0.048*love + 0.042*way + 0.032*heart + 0.028*thing + 0.027*life + 0.023*tonight + 0.021*boy + 0.018*hand


0.094*baby + 0.047*ya + 0.

In [114]:
for bow in corpus[2:80:3]:
    print bow
    print lda.get_document_topics(bow)
    print " ".join([nounid2word[e[0]] for e in bow])
    print "=========================================="

[(4160, 1), (5019, 1), (6324, 1), (4493, 1), (839, 1)]
[(0, 0.042219595632652512), (1, 0.042983946959474127), (2, 0.66285508891011624), (3, 0.25194136849775711)]


KeyError: 4160

In [109]:
bow

[(4160, 1), (5019, 1), (6324, 1), (4493, 1), (839, 1)]

In [112]:
nounid2word.keys()[1:10]

[u'5989',
 u'5982',
 u'5983',
 u'5980',
 u'5981',
 u'5986',
 u'5987',
 u'5984',
 u'5985']